In [1]:
######Import libraries
!pip install sahi
from sahi.utils.coco import Coco, CocoCategory, CocoImage, CocoAnnotation
from sahi.utils.file import save_json
from glob import glob
from sklearn.model_selection import train_test_split
from pycocotools.coco import COCO
from random import shuffle
import tensorflow as tf
import imageio
import numpy as np
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Dense, Dropout, Input, Layer
from tensorflow.keras.optimizers import SGD, Adam
import cv2
import pickle
import time
from tensorflow.keras.models import Model
from tensorflow.keras.models import model_from_json

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 8.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116934 sha256=6acd2597763a4677af40a80c9ff89a300df8cff3b9ec0b1e667a8f613fdbd6bd
  Stored in directory: /root/.cache/pip/wheels/90/d4/f7/9404e5db0116bd4d43e5666eaa3e70ab53723e1e3ea40c9a95
Successfully built fire


I used dataset from kaggle : Dataset from : https://www.kaggle.com/competitions/dogs-vs-cats/data.
The dataset containts train and test data of cat and dog images.
We can directly download Kaggle data with its respective compeition name and kaggle API token. Below is the sample code to download data directly to mounted drive.

import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/My Drive/Kaggle"
%cd /content/drive/MyDrive/Kaggle/


competition_name = "dogs-vs-cats"

# Mount your Google Drive.
from google.colab import drive
drive.mount("/content/drive")

kaggle_creds_path = "/content/drive/MyDrive/Kaggle/kaggle.json"

! pip install kaggle --quiet

! mkdir ~/.kaggle
! cp {kaggle_creds_path} ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

! kaggle competitions download -c {competition_name}

! mkdir kaggle_data
! unzip {competition_name + ".zip"} -d kaggle_data



In [2]:
###Mount drive to download data, to save pre-processed data, and model checkpoints
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Update below paths according to the location where data and models are saved.* **bold text**italicized text*
1. list_of_paths = This is used to prepare list of absolute path to train and validation dataset
2. save_path_train and save_path_valid : where data is saved in coco format
3. model_Dir = Folder to save model checkpoints
4. batch_size : batch size used in trining the model

In [3]:
"""
Update below paths
"""
list_of_paths = ["/content/drive/MyDrive/Kaggle/kaggle_data/train/cat.*.jpg",
                 "/content/drive/MyDrive/Kaggle/kaggle_data/train/dog.*.jpg"]
save_path_train = "/content/drive/MyDrive/Kaggle/kaggle_data/train_coco.json"
save_path_valid = "/content/drive/MyDrive/Kaggle/kaggle_data/valid_coco.json"
index_img_data_path = "/content/drive/MyDrive/Kaggle/kaggle_data/index_img_data_path.pkl"
model_dir = "/content/drive/MyDrive/Kaggle/kaggle_data/model_data_vgg/"
batch_size = 32

Below variables are used to control the code flow
fTrainModel : Set it to true if you want to train from scratch. If we are running for the first time, it has to be True
fSaveModel = Set it to True to save model checkpoints.
fLoadModel = Set it to true to load models from saved checkpoints
build_index = Set it to true pre-process data from scratch.

 It is must to set all the parameters to "true" when running for the first time

In [4]:
fTrainModel = True
fSaveModel = True
fLoadModel = True
build_index = True

The below code creates coco dataset using kaggle dataset.
I used tensorflow.Dataset with some optimization techniques to make it faster.
There are many helper functions to support data generator.
To speed up the experiment, I am only considering first 200 train and test images to be pre-processed and trained. (inside *get_imgs_list:* function)

In [6]:
def get_imgs_list(list_of_paths):
    """
    Parameters
    ----------
    list_of_paths : list
        DESCRIPTION: Path to source images

    Returns
    -------
    data_imgs_list : list
        DESCRIPTION: Absolute paths to all source images

    """
    data_imgs_list = []
    for path in list_of_paths:
        data_imgs_list.append(glob(path)[:200])
    return data_imgs_list

def get_label_names():
    """
    Returns
    -------
    list
        list of label names.

    """
    return ["cat", "dog"]

def get_labels():
    """
    Returns
    -------
    list
        list of labels.

    """
    return [0, 1]

def get_coco_object(labels_names, labels):
    """
    Parameters
    ----------
    labels_names : list
        DESCRIPTION : list of label names
    labels : list
        DESCRIPTION: list of labels

    Returns
    -------
    coco : Coco object
        DESCRIPTION: Add all file names to Coco object

    """
    coco = Coco()
    for label_name, label in zip(labels_names, labels):
        coco.add_category(CocoCategory(id=label, name=label_name))
    return coco

def update_coco_object(coco, list_of_imgs, labels_names, labels):
    """

    Parameters
    ----------
    coco : Coco object
        DESCRIPTION: Coco object to contain all the images
    list_of_imgs : list
        DESCRIPTION: list of images to be added to Coco object
    labels_names : list
        DESCRIPTION: list of label names
    labels : list
        DESCRIPTION: list of labels

    Returns
    -------
    coco : Coco object
        DESCRIPTION: Adding all the images to Coco object and return the same

    """
    #print("Updating coco object")
    for list_img, label, label_name in zip(list_of_imgs, labels, labels_names):
        for file in list_img:
            #All images are resized to 100, 100
            coco_image = CocoImage(file_name=file, height=100, width=100)
            coco_image.add_annotation(CocoAnnotation(bbox=[0, 0, 0, 0], category_id=label, category_name=label_name))
            coco.add_image(coco_image)
    return coco

def split_train_valid(imgs_list):
    """
    Parameters
    ----------
    imgs_list : list of list
        DESCRIPTION : list of list of image absolute paths

    Returns
    -------
    train_imgs_list : list of train image absolute paths
        DESCRIPTION: Train images path
    valid_imgs_list : list of train image absolute paths
        DESCRIPTION : Valid images path

        Split data in to train(80%) and valid(20%)
    """
    train_imgs_list = []
    valid_imgs_list = []
    num_of_train_samples = 0
    num_of_validation_samples = 0
    for li in imgs_list:
        train_li, test_li =  train_test_split(li, test_size=0.2)
        train_imgs_list.append(train_li)
        valid_imgs_list.append(test_li)
        num_of_train_samples = num_of_train_samples + len(train_li)
        num_of_validation_samples = num_of_validation_samples + len(test_li)
    return train_imgs_list, valid_imgs_list, num_of_train_samples, num_of_validation_samples

def prepare_coco_dataset(save_path_train=save_path_train,
                         save_path_valid = save_path_valid,
                         generate_test = False):
    """

    Parameters
    ----------
    save_path_train : string,
        DESCRIPTION. The default is "C:\\DuketProject\\deep-oneshot-logo-retrieval\\src\\deep_oneshot_logo_retrieval\\train_coco.json".
    save_path_valid : string
        DESCRIPTION. The default is "C:\\DuketProject\\deep-oneshot-logo-retrieval\\src\\deep_oneshot_logo_retrieval\\valid_coco.json".
    generate_test : bool, optional
        DESCRIPTION. The default is False.

    Returns
    -------
    save_path_train : string
        DESCRIPTION:  Prepare train Coco object and save in the location specified
    save_path_valid : string
        DESCRIPTION:  Prepare valid Coco object and save in the location specified

    """

    imgs_list = get_imgs_list(list_of_paths)
    if generate_test == False:
        coco = update_coco_object(get_coco_object(get_label_names(), get_labels()), imgs_list, get_label_names(), get_labels())
        save_json(data=coco.json, save_path=save_path_train)
        train_samples = 0
        valid_samples = 0
    else:
        train_list, valid_list, train_samples, valid_samples = split_train_valid(imgs_list=imgs_list)
        train_coco = update_coco_object(get_coco_object(get_label_names(), get_labels()), train_list, get_label_names(), get_labels())
        test_coco = update_coco_object(get_coco_object(get_label_names(), get_labels()), valid_list, get_label_names(), get_labels())
        save_json(data=train_coco.json, save_path=save_path_train)
        save_json(data=test_coco.json, save_path=save_path_valid)

    print("Saved coco objects")
    return save_path_train, save_path_valid, train_samples, valid_samples

#Tensorflow data generator
def get_coco_from_id(coco_id, coco, augmentation):
    """
    Parameters
    ----------
    coco_id : int
        DESCRIPTION: Reference image id from the Coco object
    coco : Coco object
        DESCRIPTION: Coco object
    augmentation : bool
        DESCRIPTION.: Augmentation to be applied to data or not

    Returns
    -------
    float32, int64
        DESCRIPTION: Reads image using img_id and label (categorical)

    """
    img_meta_data = coco.loadImgs([coco_id])[0]
    ann_ids = coco.getAnnIds(imgIds=img_meta_data['id'])
    anns = coco.loadAnns(ann_ids)
    file_name = img_meta_data['file_name']
    img = cv2.imread(file_name)
    img = cv2.resize(img, (img_meta_data['height'], img_meta_data['width']), interpolation = cv2.INTER_AREA)
    img[:,:,0] = img[:,:,0] - 103.939
    img[:,:,1] = img[:,:,1] - 116.779
    img[:,:,2] = img[:,:,2] - 123.68
    img = tf.cast(img, tf.float32)
    return img, tf.keras.utils.to_categorical(anns[0]['category_id'], num_classes=2, dtype='int64')

def python_func(idx, fc, **params):
    """
    Parameters
    ----------
    idx : int
        DESCRIPTION : imgid from COCO object
    fc : function pointer
        DESCRIPTION : Python function pointer to be called within tensorflow graph
    **params :
        DESCRIPTION :Non tensor parameters

    Returns
    -------
        DESCRIPTION : Calls and process data using python functions

    """
    outputs_types=(tf.float32, tf.int64)
    def _py_function(_idx):
        return fc(_idx, **params)
    output_tensor = tf.numpy_function(_py_function, [idx], outputs_types)
    output_tensor[0].set_shape((100, 100, 3))
    output_tensor[1].set_shape((2))
    return output_tensor

def get_coco_dataset(train_json, valid_json):
    train_coco = COCO(train_json)
    valid_coco = COCO(valid_json)
    return train_coco, valid_coco

def tf_dataset_from_coco(coco_dataset, fTrain =True):
    """
    Parameters
    ----------
    coco_dataset : COCO object
        DESCRIPTION :COCO object
    fTrain : bool, is it a train COCO object
        DESCRIPTION. The default is True.

    Returns
    -------
    dataset : tf.data.Dataset
        DESCRIPTION: tf.data.Dataset that we can feed directly to model.fit

    """
    img_ids = coco_dataset.getImgIds()
    if fTrain:
        shuffle(img_ids)
    dataset = tf.data.Dataset.from_tensor_slices(img_ids)
    if fTrain:
        dataset = (
            dataset
            .map(lambda idx: python_func(
                idx, get_coco_from_id, coco=coco_dataset, augmentation=False)
            , num_parallel_calls=tf.data.experimental.AUTOTUNE)
            .cache()
            .shuffle(buffer_size=4096)
            .repeat()
            .batch(batch_size, drop_remainder=True)
            .prefetch(tf.data.experimental.AUTOTUNE))
    else:
        dataset = (
            dataset
            .map(lambda idx: python_func(
                idx, get_coco_from_id, coco=coco_dataset, augmentation=False)
            , num_parallel_calls=tf.data.experimental.AUTOTUNE)
            .repeat()
            .batch(batch_size, drop_remainder=True)
            .prefetch(tf.data.experimental.AUTOTUNE))
    return dataset
#Get data in coco format
train_json, valid_json, train_samples, valid_samples = prepare_coco_dataset(generate_test= True)
#Get COCO objects
train_coco, valid_coco = get_coco_dataset(train_json, valid_json)
#Get tensorflow generator using coco dataset
train_ds = tf_dataset_from_coco(train_coco)
valid_ds = tf_dataset_from_coco(valid_coco, fTrain=(False))

def get_image_data(image_path):
    """
    Get image data given image path
    """
    im = cv2.imread(image_path)
    im = cv2.resize(im, (100, 100)).astype(np.float32)
    im[:,:,0] -= 103.939
    im[:,:,1] -= 116.779
    im[:,:,2] -= 123.68
    return im

def get_img_path_data():
    images_paths = get_imgs_list(list_of_paths)
    images_data = []
    images_path_data = []
    for images_path in images_paths:
        for imagePath in images_path:
            im = get_image_data(imagePath)
            images_data.append(im)
            images_path_data.append((imagePath, im))
    return images_path_data

Saved coco objects
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


I have used a VGG16 model to train a classification model.
To run the experiment quickly, I am only training for one epoch.
The basic idea of training a VGG16 model to extract features to compare with test image is taken from here : https://github.com/Tivcrmn/Keras-CNN-Image-Retrieval-System/tree/master/model

The approach suggest to use VGG features to compare two images. So when there is a test image, its extracated feature is compared with database features.

The matches that match with the test images will be retrived from database.
Hence the notebook explains how to train a model using cocodataset and how to compare extarcted features to retrieve data from database

In [7]:
if fTrainModel:
    model = VGG16(
        weights=None,
        input_shape=(100, 100, 3))

    no_soft_max_output = model.layers[-2].output
    binary_hash_output = Dense(16, activation='relu', name="hash_layer")(no_soft_max_output)#hash layer
    binary_hash_output = Dropout(0.2)(binary_hash_output)
    final_softmax_out = Dense(2, activation='softmax', name="predictions")(binary_hash_output)
    model_main = Model(inputs=model.input, outputs=final_softmax_out)
    optimizer = SGD(learning_rate=1e-3, momentum=0.9)

    model_main.compile(loss = "categorical_crossentropy", optimizer = optimizer, metrics = ['accuracy'])
    model_main.fit(train_ds, validation_data=valid_ds, steps_per_epoch=train_samples//batch_size, validation_steps=valid_samples//batch_size, epochs=1, verbose=1)

def save_model(model_name, mod_model):

    model_json = mod_model.to_json()
    with open(model_dir + model_name + ".json", "w") as json_file:
        json_file.write(model_json)

    mod_model.save_weights(model_dir + model_name + ".h5")
    print("Saved {0} model to disk".format(model_name))

def load_model(model_name):
    json_file = open(model_dir + model_name + '.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()

    model = model_from_json(loaded_model_json)
    # load weights into new model
    model.load_weights(model_dir + model_name + ".h5")
    print("Loaded model {0} from disk".format(model_name))
    return model

if fSaveModel:
    save_model("model_orig", model_main)

    ## two layers removed for obtaining binary hash layer output
    bin_hash_output = model_main.layers[-3].output
    model_pop2 = Model(inputs=model_main.input, outputs=bin_hash_output)
    save_model("model_pop2", model_pop2)
    ## two layers removed for obtaining feature layer output
    feature_output = model_main.layers[-5].output
    model_pop4 = Model(inputs=model_main.input, outputs=feature_output)
    save_model("model_pop4", model_pop4)

if fLoadModel:
    model_orig = load_model("model_pop4")
    model_bh = load_model("model_pop2")


if build_index == True:
    index = {}
    images_path_data = get_img_path_data()
    for image_path, image_data in images_path_data:
        im = np.expand_dims(image_data, axis=0)
        pred = model_orig.predict(im)
        pred_bh = model_bh.predict(im)

        index[image_path] = [pred_bh, pred]
        if(len(index.keys())%100)==1:
            print ("Total images done: {0}/{1}".format(len(index.keys()), len(images_path_data)))

    print("we are now done indexing our image -- now saving the index to disk...")
    index_dir = index_img_data_path
    with open(index_dir, "wb") as f:
        pickle.dump(index, f)


def get_pickle_index():
    arg = index_img_data_path
    with open(arg, "rb") as f:
        index = pickle.load(f, encoding='latin1')

    print("binary hash classes: ", len(index.keys()))
    return index

def get_img_lbl_map():
    imgs_lists = get_imgs_list(list_of_paths)
    labels = get_labels()
    map_path_label = {}

    for ind, imgs_list in enumerate(imgs_lists):
        image_label = labels[ind]
        for image_path in (imgs_list):
            map_path_label[image_path] = image_label
    return map_path_label


class Searcher:
    def __init__(self, index, map_path_label, limit=10):
        # store our index path
        self.index = index
        self.limit = limit
        self.map_path_label = map_path_label
        self.hash_str = lambda x: '1' if x==1 else '0'

    def chi2_distance(self, histA, histB, eps = 1e-10):
        d = 0.5 * np.sum([((a - b) ** 2) / (a + b + eps)
            for (a, b) in zip(histA, histB)])
        return d

    def hamming_distance(self, a, b):
        cnt=0
        for i ,j in zip(a, b):
            if i!=j:
                cnt+=1
        return cnt

    def search(self, image_path, use_hash = False, hamming_distance = 1):
        t1 = time.time()

        results, acc = self.search_without_hash(image_path)
        t2 = time.time()
        time_diff = t2-t1

        return (results, time_diff, acc)

    def search_without_hash(self, image_path):
        # initialize our dictionary of results
        results = {}
        query_label = self.map_path_label[image_path]
        image_data = get_image_data(image_path)
        queryFeatures = model_orig.predict(np.expand_dims(image_data, axis=0))
        #queryFeatures = index[image_path][1]

        for key in self.index.keys():
            images_path = self.index[key]
            #for image_path, feature in images_path:

            d = self.chi2_distance(images_path[1], queryFeatures)
            results[image_path] = (key, d)

        results = sorted([(v[1], v[0], k) for (k,v) in results.items()])
        results = results[:self.limit]

        acc_score = 0
        for item in results:
            item_label = self.map_path_label[item[2]]
            if item_label==query_label:
                acc_score += 1

        acc_score = (acc_score*100.0)/float(self.limit)
        return results, acc_score

no_queries = 40
no_retrieve = 20
hamming_distance = 2

searcher = Searcher(index=get_pickle_index(), map_path_label=get_img_lbl_map(), limit=no_retrieve)

[cat_img_paths, dog_img_paths] = get_imgs_list(list_of_paths)


map_path_label = get_img_lbl_map()
for image_path in cat_img_paths[:3]:
    query_label = map_path_label[image_path]
    results1 = searcher.search(image_path, use_hash = False)

    print(results1)

10/10 [==============================] - 350s 19s/step - loss: 0.7240 - accuracy: 0.4844 - val_loss: 0.6820 - val_accuracy: 0.6250
Saved model_orig model to disk
Saved model_pop2 model to disk
Saved model_pop4 model to disk
Loaded model model_pop4 from disk
Loaded model model_pop2 from disk
1/1 [==============================] - 0s 481ms/step
Total images done: 1/400
1/1 [==============================] - 0s 243ms/step
Total images done: 101/400
1/1 [==============================] - 0s 135ms/step
Total images done: 201/400
1/1 [==============================] - 0s 141ms/step
Total images done: 301/400
1/1 [==============================] - 0s 238ms/step
we are now done indexing our image -- now saving the index to disk...
binary hash classes:  400
1/1 [==============================] - 0s 443ms/step
([(21.736539840698242, '/content/drive/MyDrive/Kaggle/kaggle_data/train/dog.9688.jpg', '/content/drive/MyDrive/Kaggle/kaggle_data/train/cat.9076.jpg')], 1.8514792919158936, 5.0)
1/1 [=====

Results printed at the end shows the list of images matched to the test images.
We need to train VGG16 for better performance and then test for good results.
As test images, I am using "cat" labeled images and trying to retrieve cat images from database.